## Utilization Prediction

### Import Libraries

In [114]:
import pandas as pd
import numpy as np

import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

%matplotlib inline

### Import Data

In [ ]:
charging_sessions_data = pd.read_csv("cleaned_data.csv")
weather_data = pd.read_csv("cleaned_weather_data.csv")

charging_sessions_data

In [ ]:
charging_sessions_data.info()

In [117]:
# Due to the export to csv file the right format cannot be kept
# Therefore, the columns with datetime type need to be reformatted
charging_sessions_data["connectionTime"] = pd.to_datetime(charging_sessions_data["connectionTime"])
charging_sessions_data["disconnectTime"] = pd.to_datetime(charging_sessions_data["disconnectTime"])
charging_sessions_data["doneChargingTime"] = pd.to_datetime(charging_sessions_data["doneChargingTime"])
weather_data["timestamp"] = pd.to_datetime(weather_data["timestamp"])

### Developing Prediction Model on hourly Utilization -> connectiontime to disconnectiontime in hours
1. **Feature Engineering**
* Normalizing and standardizing data **(Angela)**
* Feature encoding (transforming categorical values into numerical values) **(Coco)**
* Determining features (Correlation) **(Marietta)**
* Join charging sessions and weather data **(Coco)**
2. **Find Optimal Machine Learning Method**
* Test different ML methods and evaluate them based on one metrics
    * Polynomial Regression **(Simon)**
    * Lasso Regression **(Simon)**
    * Ridge Regression **(Marietta)**
    * Random Forest **(Angela)**
    * Neural Network **(Coco)**
* Choose the ML methods demonstrating the best performance (Ridge Regression)
3. **Developing Predictive Models**
* Prediction models are developed for its charging site
* Develop predictive model using neural networks
* Develop predictive model using any other machine learning methods of choice (see Section 1.)
* Use cross-validation to train the models
* Compare predictive performance of both models on the same holdout set
* Determine the type of model the operator should employ
4. **Examples for Business Case**
* Visualize data prediction to support/ enables business case
* Make example predictions to support/ enables business case

### 1. Feature Engineering

#### 1.1. Data Standardization and Normalization

Standardizing and normalizing the data is important to improve the model performance since most models assume that the features are on a similar scale. The choice between standardization and normalization is dependent on the machine learning model. \
We have decided to standardize the data for Polynomial Regression, Lasso and Ridge Regression. Neural Networks can use either options, but normalization is often preferred. Thus, we will apply normalization to its data. \
Random Forest and other tree-based models are inherently insensitive to the scale of the features, so scaling the dataset does not impact performance. Therefore its training data does not require any scaling. \
It is important to note that the data should be scaled after splitting the dataset into training and test data to prevent data leakage and preserve the validity of the test results. Thus, this part demonstrate how data can be scaled accordingly.

##### 1.1.1. Charging Session Data Standardization and Normalization

In [ ]:
charging_sessions_data.dtypes

In [ ]:
charging_sessions_data.columns

In [ ]:
# drop non-numerical data
scaled_charging_sessions_data = charging_sessions_data[['kWhDelivered', 'WhPerMile',
       'kWhRequested', 'milesRequested', 'minutesAvailable']]
scaled_charging_sessions_data.describe().round(3)

In [ ]:
# X1 is for standardized data and X2 is for normalized data
X1 = scaled_charging_sessions_data
X2 = scaled_charging_sessions_data

# standardize data
scaleStandard = StandardScaler()
X1 = scaleStandard.fit_transform(X1)
X1 = pd.DataFrame(X1, columns=['kWhDelivered', 'WhPerMile', 'kWhRequested', 'milesRequested', 'minutesAvailable'])
X1.describe().round(3)

In [ ]:
# normalize data
scaleMinMax = MinMaxScaler(feature_range=(0, 1))
X2 = scaleMinMax.fit_transform(X2)
X2 = pd.DataFrame(X2, columns=['kWhDelivered', 'WhPerMile', 'kWhRequested', 'milesRequested', 'minutesAvailable'])
X2.describe().round(3)

##### 1.1.2. Weather Data Standardization and Normalization

In [ ]:
weather_data.dtypes

In [ ]:
weather_data.columns

In [ ]:
# drop non-numerical data
scaled_weather_data = weather_data[['temperature', 'cloud_cover', 'pressure', 'windspeed', 'precipitation',
       'felt_temperature']]
scaled_charging_sessions_data.describe().round(3)

In [ ]:
# X1 is for standardized data and X2 is for normalized data
X11 = scaled_weather_data
X22 = scaled_weather_data

# standardize data
scaleStandard = StandardScaler()
X11 = scaleStandard.fit_transform(X11)
X11 = pd.DataFrame(X11, columns=['temperature', 'cloud_cover', 'pressure', 'windspeed', 'precipitation',
       'felt_temperature'])
X11.describe().round(3)

In [ ]:
# normalize data
scaleMinMax = MinMaxScaler(feature_range=(0, 1))
X22 = scaleMinMax.fit_transform(X22)
X22 = pd.DataFrame(X22, columns=['temperature', 'cloud_cover', 'pressure', 'windspeed', 'precipitation',
       'felt_temperature'])
X22.describe().round(3)

#### 1.2. Feature Enconding

In [128]:
# Encoding weekday 
weather_data["is_weekday"] = np.where(weather_data['timestamp'].dt.weekday.isin([5, 6]), 0, 1)
charging_sessions_data["is_weekday"] = np.where(charging_sessions_data['connectionTime'].dt.weekday.isin([5, 6]), 0, 1)

# Encoding siteID
charging_sessions_data["siteID"] = np.where(charging_sessions_data['siteID'] == 1, 1, 0)

# Extract hour and month
weather_data["month"] = weather_data['timestamp'].dt.month
weather_data["hour"] = weather_data['timestamp'].dt.hour
charging_sessions_data["month"] = charging_sessions_data['connectionTime'].dt.month
charging_sessions_data["hour"] = charging_sessions_data['connectionTime'].dt.hour

# Encoding hour by Sin-Cos Encoding
charging_sessions_data['hour_sin'] = np.sin(2 * np.pi * charging_sessions_data['hour'] / 24)
charging_sessions_data['hour_cos'] = np.cos(2 * np.pi * charging_sessions_data['hour'] / 24)

# Encoding month by Sin-Cos Encoding

charging_sessions_data['month_sin'] = np.sin(2 * np.pi * charging_sessions_data['month'] / 12)
charging_sessions_data['month_cos'] = np.cos(2 * np.pi * charging_sessions_data['month'] / 12)

In [ ]:
weather_data.columns

In [ ]:
# Lets group the table by month, hour and weekday to later join it in the charging data
# Therefore other attributes have to be averaged 
grouped_weather = ( weather_data
.groupby(['month', 'is_weekday', 'hour'])
.agg({
    'temperature': 'mean',   # Average temperature
    'precipitation': 'mean',      # Average precipitation
    'windspeed': 'mean', # Average windspeed
    'cloud_cover': 'mean', # Average cloud cover
    'pressure': 'mean'    # Average pressure
}).reset_index())
grouped_weather.rename(columns = {"temperature" : "avg_temperature", "precipitation": "avg_precipitation","windspeed":"avg_windspeed","cloud_cover":"avg_cloud","pressure":"avg_pressure"}, inplace = True)
grouped_weather

#### 1.3. Join charging sessions and weather data

In [131]:
# Now we merge weather data to charging session data on month, hour and is_weekday

merged_sessions = charging_sessions_data.merge(grouped_weather, on = ["month", "is_weekday", "hour"] )
merged_sessions = merged_sessions.drop(['spaceID','stationID','timezone','userID', 'userInputs'], axis = 1 )

Now that the session data contain all the recorded charging sessions with an addition of weather information, we have to think about how to map these to the target variable Utilization (each entry has to have a target variable)

In [ ]:
merged_sessions

In [ ]:
merged_sessions.columns

#### 1.4. Determining Features (Correlation)

In [ ]:
charging_sessions_data.columns

In [ ]:
# choice of relecant numeric features 
correlation_features = [
    'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'is_weekday',
    'avg_temperature', 'avg_windspeed',  'avg_precipitation', 'avg_pressure', 'avg_cloud',
    'WhPerMile', 'kWhRequested', 'milesRequested', 'minutesAvailable', 'doneChargingTime'  
]

# calculate the correlation 
correlation_matrix = merged_sessions[correlation_features].corr()

# plot a heatmap of the correlation 
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix: Charging Sessions and Weather Data')
plt.show()
### 2. Find Optimal Machine Learning Method

### 2. Find Optimal Machine Learning Method

#### 2.1. Predicting Utilization Value

In [ ]:
# count the stations for each siteID 
stations_per_site = charging_sessions_data.groupby('siteID')['stationID'].nunique().reset_index()
stations_per_site.rename(columns={'stationID': 'total_stations'}, inplace=True)

print(stations_per_site)

In [ ]:
# Calculate the duration of each session
charging_sessions_data['connectionTime'] = pd.to_datetime(charging_sessions_data['connectionTime'])
charging_sessions_data['disconnectTime'] = pd.to_datetime(charging_sessions_data['disconnectTime'])
charging_sessions_data['duration'] = (charging_sessions_data['disconnectTime'] - charging_sessions_data['connectionTime']).dt.total_seconds() / 3600  # duration in hours

# Summarise the loading times per hour and site
hourly_utilization = charging_sessions_data.groupby(['connectionTime', 'siteID']).agg(
    active_time=('duration', 'sum'),  # sum of the loading time in hours
).reset_index()

# Add the total number of stations for each site
hourly_utilization = hourly_utilization.merge(stations_per_site, on='siteID', how='left')

# Calculate the utilization per hour
hourly_utilization['utilization'] = hourly_utilization['active_time'] / (hourly_utilization['total_stations'])

#Check the first five results
print(hourly_utilization)

# prediciting utilization value

In [ ]:
merged_sessions = merged_sessions.merge(
    hourly_utilization[['connectionTime', 'siteID', 'utilization']],  
    on=['connectionTime', 'siteID'],  # same columsn for the merge
    how='left'  # all columns from charging session data should remain 
)
# check
merged_sessions.head()

In [ ]:
merged_sessions.columns

In [ ]:
# choice of relevant numeric features 
correlation_features = [
    'hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'is_weekday',
    'avg_temperature', 'avg_windspeed',  'avg_precipitation', 'avg_pressure', 'avg_cloud',
    'WhPerMile', 'kWhRequested', 'milesRequested', 'minutesAvailable', 'doneChargingTime',
    'session_duration', 'idle_time', 'energy_ratio', 'kWhDelivered', 'utilization' 
]

# calculate the correlation 
correlation_matrix = merged_sessions[correlation_features].corr()

# plot a heatmap of the correlation 
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix: Charging Sessions and Weather Data')
plt.show()
### 2. Find Optimal Machine Learning Method

In [ ]:
merged_sessions.info()

#### 2.2. Polynomial Regression

In [ ]:
# Make a copy (not strictly required, but good practice)
merged_sessions_poly = merged_sessions.copy()

# Drop rows with NaN in session_duration, idle_time, or utilization
cols_needed = ['session_duration', 'idle_time', 'utilization']
merged_sessions_poly = merged_sessions_poly.dropna(subset=cols_needed)

# Select features and target
X_poly = merged_sessions_poly[['session_duration', 'idle_time']]
y_poly = merged_sessions_poly['utilization']

# 60/20/20 split
X_train, X_temp, y_train, y_temp = train_test_split(
    X_poly, 
    y_poly, 
    test_size=0.4,   # 40% for test+holdout
    random_state=42
)
X_test, X_holdout, y_test, y_holdout = train_test_split(
    X_temp, 
    y_temp, 
    test_size=0.5,   # split temp 50/50 => 20/20
    random_state=42
)

# Create polynomial features (degree=2, no bias)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
X_holdout_poly = poly.transform(X_holdout)

# Fit a linear model on the polynomial-transformed features
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)

# Predict on train, test, holdout
y_pred_train = poly_model.predict(X_train_poly)
y_pred_test = poly_model.predict(X_test_poly)
y_pred_holdout = poly_model.predict(X_holdout_poly)

# Evaluate metrics for train, test, holdout
train_r2 = r2_score(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)

test_r2 = r2_score(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

holdout_r2 = r2_score(y_holdout, y_pred_holdout)
holdout_mse = mean_squared_error(y_holdout, y_pred_holdout)

# Print results
print("Polynomial Regression with 'session_duration' and 'idle_time' -> Target: 'utilization'")
print(f"Train R²: {train_r2:.4f}")
print(f"Train MSE: {train_mse:.6f}")
print(f"Test R²:  {test_r2:.4f}")
print(f"Test MSE:  {test_mse:.6f}")
print(f"Holdout R²: {holdout_r2:.4f}")
print(f"Holdout MSE: {holdout_mse:.6f}")


#### 2.3. Lasso Regression

In [ ]:
# Make a copy of your DataFrame (optional, but good practice)
merged_sessions_lasso = merged_sessions.copy()

# Drop rows that have NaN in any relevant columns:
#    We need 'session_duration', 'idle_time', AND 'utilization' to be valid
needed_cols = ['session_duration', 'idle_time', 'utilization']
merged_sessions_lasso = merged_sessions_lasso.dropna(subset=needed_cols)

# Define features (X) and target (y)
X_lasso = merged_sessions_lasso[['session_duration', 'idle_time']]
y_lasso = merged_sessions_lasso['utilization']

# Split into train (60%), test (20%), holdout (20%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_lasso, 
    y_lasso, 
    test_size=0.4,  # 40% for test+holdout
    random_state=42
)

X_test, X_holdout, y_test, y_holdout = train_test_split(
    X_temp, 
    y_temp, 
    test_size=0.5,  # split temp into 20% / 20%
    random_state=42
)

# Standardize (fit on training, transform all)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_holdout_scaled = scaler.transform(X_holdout)

# Perform Lasso + GridSearch for alpha
lasso = Lasso(max_iter=10000, random_state=42)

#    alpha from 1e-4 to 10, ~50 values
param_grid = {'alpha': np.logspace(-4, 1, 50)}  

grid_search = GridSearchCV(
    estimator=lasso, 
    param_grid=param_grid, 
    scoring='r2', 
    cv=5,
    n_jobs=-1
)
grid_search.fit(X_train_scaled, y_train)

# Retrieve best alpha
best_alpha = grid_search.best_params_['alpha']
print("Best Alpha:", best_alpha)

# Fit final Lasso model
best_lasso = Lasso(alpha=best_alpha, max_iter=10000, random_state=42)
best_lasso.fit(X_train_scaled, y_train)

# Predictions
y_pred_train = best_lasso.predict(X_train_scaled)
y_pred_test = best_lasso.predict(X_test_scaled)
y_pred_holdout = best_lasso.predict(X_holdout_scaled)

# Evaluate on Train, Test, Holdout
train_r2 = r2_score(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)

test_r2 = r2_score(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

holdout_r2 = r2_score(y_holdout, y_pred_holdout)
holdout_mse = mean_squared_error(y_holdout, y_pred_holdout)

print("Lasso Regression with 'session_duration' and 'idle_time' -> 'utilization'")
print(f"Best Alpha: {best_alpha}")
print(f"Train R²: {train_r2:.4f} | MSE: {train_mse:.6f}")
print(f"Test  R²: {test_r2:.4f}  | MSE: {test_mse:.6f}")
print(f"Holdout R²: {holdout_r2:.4f} | MSE: {holdout_mse:.6f}")




#### 2.4. Ridge Regression

In [ ]:
# only keep numeric data for the regression
merged_sessionsRidge = merged_sessions.select_dtypes(include=['number']) 
print(merged_sessionsRidge.dtypes)

In [145]:
# Select Features and Target
# Define target variable
target = 'utilization'

# Define features 
XRidge = merged_sessionsRidge[['session_duration', 'idle_time']] # features
yRidge = merged_sessionsRidge['utilization'] # target variable

# split dataset into training (60%), testing (20%) and holdout (20%) sets
#X_train, X_test, y_train, y_test = train_test_split(XRidge, yRidge, test_size=0.3, random_state=42)
X_train, X_temp, y_train, y_temp = train_test_split(XRidge, yRidge, test_size=0.4, random_state=42)  # 40% temp (test + holdout)
X_test, X_holdout, y_test, y_holdout = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # Split temp 50/50

# scale
scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_holdout_scaled = scaler.transform(X_holdout)

In [ ]:
# Ridge-Regression Model
ridge = Ridge(max_iter=10000, random_state=42)

# Set up a grid search for Ridge with different alpha values
param_grid = {'alpha': np.logspace(-4, 1, 50)}  # Test alpha values from 0.0001 to 10
grid_search = GridSearchCV(ridge, param_grid, scoring='r2', cv=5)
grid_search.fit(X_train_scaled, y_train)

# Best alpha value
best_alphaRidge = grid_search.best_params_['alpha']
print(f"Best Alpha: {best_alphaRidge}")

# Train the Ridge model with the best alpha
best_ridge = Ridge(alpha=best_alphaRidge, max_iter=10000, random_state=42)
best_ridge.fit(X_train_scaled, y_train)

# Predictions
y_pred_train = best_ridge.predict(X_train_scaled)
y_pred_test = best_ridge.predict(X_test_scaled)
y_pred_holdout = best_ridge.predict(X_holdout_scaled)

In [ ]:
# Evaluation of the model
# Evaluation on the training set
train_r2 = r2_score(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)

# Evaluation on the test set
test_r2 = r2_score(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

#Evaluation on the holdout set
holdout_r2 = r2_score(y_holdout, y_pred_holdout)
holdout_mse = mean_squared_error(y_holdout, y_pred_holdout)

# print the results
print("Train R²:", train_r2)
print("Train MSE:", train_mse)
print("Test R²:", test_r2)
print("Test MSE:", test_mse)
print("Holdout R²:", holdout_r2)
print("Holdout MSE:", holdout_mse)

The model performs well, with high R² values on Train (92.7%), Test (91.7%), and Holdout (89.4%) sets, indicating strong explanatory power and good generalization. 
The MSE values are consistently low, reflecting accurate predictions across all datasets. The close performance between Test and Holdout suggests the model is not overfitted and is robust for unseen data.

In [ ]:
#Visualization
# Plot predicted vs actual for test data
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred_test, alpha=0.7, label="Test Data")
plt.scatter(y_holdout, y_pred_holdout, alpha=0.7, label="Holdout Data")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', label="Perfect Prediction")
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs Predicted")
plt.legend()
plt.show()

#### 2.5. Random Forest

In [ ]:
merged_session_rf = merged_sessions

# Select Features and Target
# Define target variable
target = 'utilization'

# Define features
X_rf = merged_session_rf[['session_duration', 'idle_time']] # features
y_rf = merged_session_rf['utilization'] # target variable

# Split dataset into training, testing and holdout sets
# X_train, X_test, y_train, y_test = train_test_split(X_rf, y_rf, test_size=0.4, random_state=42)
X_train, X_temp, y_train, y_temp = train_test_split(X_rf, y_rf, test_size=0.4, random_state=42)  # 40% temp (test + holdout)
X_test, X_holdout, y_test, y_holdout = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # Split temp 50/50

# Train model on 100 random trees on train set
rf_model = RandomForestRegressor(n_estimators=100, bootstrap=True, random_state=42) # we select boostrapp, i.e. we use bagging
rf_model.fit(X_train,y_train)

In [ ]:
# Predictions
y_pred_train = rf_model.predict(X_train)
y_pred_test = rf_model.predict(X_test)
y_pred_holdout = rf_model.predict(X_holdout)

# evaluate performance based on metrics on train and test set
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred) # good performance = close to 1

# Evaluation on the training set
train_r2 = r2_score(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)

# Evaluation on the test set
test_r2 = r2_score(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

#Evaluation on the holdout set
holdout_r2 = r2_score(y_holdout, y_pred_holdout)
holdout_mse = mean_squared_error(y_holdout, y_pred_holdout)

print("Train R²:", train_r2)
print("Train MSE:", train_mse)
print("Test R²:", test_r2)
print("Test MSE:", test_mse)
print("Holdout R²:", holdout_r2)
print("Holdout MSE:", holdout_mse)

In [ ]:
# hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200], # no of trees
    'max_depth': [5, 10, 20], # depth of each tree
    'min_samples_split': [1, 2, 5], # no required to split
    'min_samples_leaf': [1, 2, 4] # min samples required to node
}

# training of holdout set
rf_model_cv = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
rf_model_cv.fit(X_holdout, y_holdout)

In [ ]:
print("Best hyperparameters:", rf_model_cv.best_params_)

In [ ]:
# Predictions
y_pred_train = rf_model_cv.predict(X_train)
y_pred_test = rf_model_cv.predict(X_test)
y_pred_holdout = rf_model_cv.predict(X_holdout)

# evaluate performance based on metrics on train and test set
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred) # good performance = close to 1

# Evaluation on the training set
train_r2 = r2_score(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)

# Evaluation on the test set
test_r2 = r2_score(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

#Evaluation on the holdout set
holdout_r2 = r2_score(y_holdout, y_pred_holdout)
holdout_mse = mean_squared_error(y_holdout, y_pred_holdout)

print("Train R²:", train_r2)
print("Train MSE:", train_mse)
print("Test R²:", test_r2)
print("Test MSE:", test_mse)
print("Holdout R²:", holdout_r2)
print("Holdout MSE:", holdout_mse)

#### 2.5. Neural network

The MLPRegressor (Multi-Layer Perceptron Regressor) in scikit-learn is a neural network model that implements a feedforward artificial neural network with fully connected layers, trained using backpropagation.

For this method we decided to include more features 

In [ ]:
from sklearn.neural_network import MLPRegressor

# Conduct train test split
from sklearn.model_selection import train_test_split

X = merged_sessions[["siteID","WhPerMile","kWhRequested","minutesAvailable","is_weekday","session_duration","idle_time","avg_temperature","avg_precipitation","avg_windspeed", "hour_sin", "hour_cos"]]
y = merged_sessions['utilization']

# Split into train, validation and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=123)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=123)

# Scale train data
NN_features = ["siteID","WhPerMile","kWhRequested","minutesAvailable","is_weekday","session_duration","idle_time","avg_temperature","avg_precipitation","avg_windspeed", "hour_sin", "hour_cos"]
X_NN_scale = X_train[["siteID","WhPerMile","kWhRequested","minutesAvailable","is_weekday","session_duration","idle_time","avg_temperature","avg_precipitation","avg_windspeed"]]
X_NN_exclude = X_train[["hour_sin","hour_cos"]]

# Scaling the chosen deatures except for hour_sin and hour_cos (these are naturally in scale)
X_NN_scaled = scaleStandard.fit_transform(X_NN_scale)
X_train = pd.DataFrame(
    np.hstack([X_NN_scaled, X_NN_exclude.values]),  # Combine scaled and unscaled features
    columns=NN_features # Keep the column names
)

# scale test data
X_NN_scale = X_val[["siteID","WhPerMile","kWhRequested","minutesAvailable","is_weekday","session_duration","idle_time","avg_temperature","avg_precipitation","avg_windspeed"]]
X_NN_exclude = X_val[["hour_sin","hour_cos"]]

# Scaling the chosen deatures except for hour_sin and hour_cos (these are naturally in scale)
X_NN_scaled = scaleStandard.fit_transform(X_NN_scale)
X_val = pd.DataFrame(
    np.hstack([X_NN_scaled, X_NN_exclude.values]),  # Combine scaled and unscaled features
    columns=NN_features # Keep the column names
)

# scale holdout data
X_NN_scale = X_test[["siteID","WhPerMile","kWhRequested","minutesAvailable","is_weekday","session_duration","idle_time","avg_temperature","avg_precipitation","avg_windspeed"]]
X_NN_exclude = X_test[["hour_sin","hour_cos"]]

# Scaling the chosen deatures except for hour_sin and hour_cos (these are naturally in scale)
X_NN_scaled = scaleStandard.fit_transform(X_NN_scale)
X_test = pd.DataFrame(
    np.hstack([X_NN_scaled, X_NN_exclude.values]),  # Combine scaled and unscaled features
    columns=NN_features # Keep the column names
)

param_grid = {
    'hidden_layer_sizes': [(100,), (100,50), (100, 50)],  # Different layer sizes
    'activation': ['relu', 'tanh'],  # Different activation functions
    'learning_rate_init': [0.001, 0.01, 0.1],  # Different learning rates
}
mlp = MLPRegressor(
    max_iter=100,     
)

# Grid search with 5-fold cross-validation on the training set
grid_search = GridSearchCV(mlp, param_grid, scoring='r2', cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

In [ ]:
# Initialize the hyperparameters with the best parameters
best_mlp = MLPRegressor(
    hidden_layer_sizes=best_params['hidden_layer_sizes'],
    activation=best_params['activation'],
    learning_rate_init=best_params['learning_rate_init'],
    max_iter=100,
    random_state=42
)
best_mlp.fit(X_train, y_train)

# Predictions
y_pred_train = best_mlp.predict(X_train)
y_pred_val = best_mlp.predict(X_val)

# Evaluate the model
train_r2 = r2_score(y_train, y_pred_train)
val_r2 = r2_score(y_val, y_pred_val)
train_mse = mean_squared_error(y_train, y_pred_train)
val_mse = mean_squared_error(y_val, y_pred_val)

print("1. Validation")
print("Train R²:", train_r2)
print("Train MSE:", train_mse)
print("Validation R²:", val_r2)
print("Validation MSE:", val_mse)

# Tuning around best parameters
fine_tuned_param_grid = {
    'hidden_layer_sizes': [(100,10),(100,20),(100,30),(100,40),(100,50),(100,60),(100,70)],
    'learning_rate_init': [best_params['learning_rate_init'] / 2, best_params['learning_rate_init'], best_params['learning_rate_init'] * 2],
}
grid_search_fine = GridSearchCV(mlp, fine_tuned_param_grid, scoring='r2', cv=5)
grid_search_fine.fit(X_train, y_train)
better_params = grid_search_fine.best_params_
print(f"Fine-Tuned Best Parameters: {better_params}")

tuned_mlp = MLPRegressor(
    hidden_layer_sizes=better_params['hidden_layer_sizes'],
    activation=best_params['activation'],
    learning_rate_init=better_params['learning_rate_init'],
    max_iter=100,
    random_state=42
)

tuned_mlp.fit(X_train, y_train) 

# Predictions
y_pred_train = tuned_mlp.predict(X_train)
y_pred_val = tuned_mlp.predict(X_val)
train_r2 = r2_score(y_train, y_pred_train)
val_r2 = r2_score(y_val, y_pred_val)
train_mse = mean_squared_error(y_train, y_pred_train)
val_mse = mean_squared_error(y_val, y_pred_val)

print("2. Validation")
print("Train R²:", train_r2)
print("Train MSE:", train_mse)
print("Validation R²:", val_r2)
print("Validation MSE:", val_mse)

In [ ]:
# Final evaluation
y_pred_test = tuned_mlp.predict(X_test)
test_r2 = r2_score(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)
print("Holdout R²:", test_r2)
print("Holdout MSE:", test_mse)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_val, y_pred_val , alpha=0.7, label="Testing Data")
plt.scatter(y_test, y_pred_test, alpha=0.7, label="Holdout Data")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', label="Perfect Prediction")

plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Neural Network: Actual vs Predicted")
plt.legend()
plt.show()

### 3. Developing Predictive Models

#### 3.1. Train Models

#### 3.2. Cross-Validation

#### 3.3. Performance Evaluation

### 4. Examples for Business Case

#### 4.1. Visualizing Prediction Data

In [ ]:

# Simulate predicted utilization data for a single site over 24 hours
#    In a real scenario, you'd have actual predictions from your trained model.
np.random.seed(42)
hours = np.arange(24)
# Let's assume predicted utilization values range between 0.0 and 1.0 (0% to 100%)
predicted_utilization = np.random.normal(loc=0.5, scale=0.2, size=24)
predicted_utilization = np.clip(predicted_utilization, 0, 1)  # Ensure within [0,1]

# Define a function to decide dynamic pricing based on predicted utilization
#    For instance, if utilization >= 70%, we set a "peak" price, else an "off-peak" price.
def determine_price(util):
    if util >= 0.7:
        return 0.40  # Example: 0.40 $/kWh at peak
    else:
        return 0.25  # Example: 0.25 $/kWh off-peak

prices = [determine_price(u) for u in predicted_utilization]

# Combine into a DataFrame for easy manipulation
data = {
    "hour": hours,
    "predicted_utilization": predicted_utilization,
    "price_per_kWh": prices
}
df = pd.DataFrame(data)

# Show how the price changes based on predicted utilization
#    We’ll mark hours that exceed a threshold (e.g., 70%).
peak_threshold = 0.7
df["is_peak"] = df["predicted_utilization"] >= peak_threshold

# Visualize in a bar chart: hour vs. predicted utilization
fig, ax1 = plt.subplots(figsize=(12,6))

# Plot utilization bars
colors = df["is_peak"].map({True: "salmon", False: "skyblue"})
ax1.bar(df["hour"], df["predicted_utilization"], color=colors, alpha=0.7, label="Predicted Utilization")
ax1.set_xlabel("Hour of the Day")
ax1.set_ylabel("Predicted Utilization (0.0–1.0)")
ax1.set_ylim(0, 1.1)
ax1.set_xticks(range(24))
ax1.set_title("Predicted Hourly Utilization & Dynamic Pricing")

# We'll add a horizontal line to indicate the threshold
ax1.axhline(y=peak_threshold, color="red", linestyle="--", alpha=0.5, label="Peak Threshold (0.7)")

# Plot a second axis for the Price over the same hours
ax2 = ax1.twinx()
ax2.plot(df["hour"], df["price_per_kWh"], color="green", marker="o", label="Dynamic Price/kWh")
ax2.set_ylabel("Price ($/kWh)")

# Build a combined legend
h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
ax1.legend(h1 + h2, l1 + l2, loc="upper left")

plt.show()

# Print out the DataFrame for clarity
print("Hourly Predicted Utilization & Dynamic Price:\n", df)


#### 4.2. Example Prediction